# Imports + class

In [1]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from z3 import *
from xgboost import XGBClassifier
from pmlb import fetch_data
set_option(rational_to_decimal=True)

import re
import time
import pandas as pd
from model import XGBoostExplainer, generate_results
from sklearn.model_selection import train_test_split
np.set_printoptions(suppress=True)

In [2]:
class XGBoostExplainer:
    def __init__(self, model, data):
        self.model = model
        self.data = data.values
        self.columns = data.columns
        self.max_categories = 2

        set_option(rational_to_decimal=True)
        self.categoric_features = self.get_categoric_features(self.data)
        self.T_model = self.model_trees_expression(self.model)
        self.T = self.T_model

    def explain(self, instance, reorder="asc"):
        self.I = self.instance_expression(instance)
        self.D, self.D_add = self.decision_function_expression(self.model, [instance])
        return self.explain_expression(self.I, And(self.T, self.D_add), self.D, self.model, reorder)

    def explain_prob(self, instance, reorder="asc", threshold_margin=0, target_threshold=None):
        self.I = self.instance_expression(instance)
        self.D, self.D_add = self.decision_function_expression(self.model, [instance])
        return self.explain_expression_prob(self.I, And(self.T, self.D_add), self.D, self.model, reorder, threshold_margin, target_threshold)

    def get_categoric_features(self, data: np.ndarray):
        categoric_features = []
        for i in range(data.shape[1]):
            feature_values = data[:, i]
            unique_values = np.unique(feature_values)
            if len(unique_values) <= self.max_categories:
                categoric_features.append(self.columns[i])

        return categoric_features

    def feature_constraints(self, constraints=[]):
        """TODO
        esperado receber limites das features pelo usuário
        formato previso: matriz/dataframe [feaature, min/max, valor]
        constraaint_expression = "constraaint_df_to_feature()"
        """
        return

    def model_trees_expression(self, model):
        df = model.get_booster().trees_to_dataframe()
        if model.get_booster().feature_names == None:
            feature_map = {f"f{i}": col for i, col in enumerate(self.columns)}
            df["Feature"] = df["Feature"].replace(feature_map)

        df["Split"] = df["Split"].round(4)
        self.booster_df = df
        class_index = 0  # if model.n_classes_ == 2:
        all_tree_formulas = []

        for tree_index in df["Tree"].unique():
            tree_df = df[df["Tree"] == tree_index]
            o = Real(f"o_{tree_index}_{class_index}")

            if len(tree_df) == 1 and tree_df.iloc[0]["Feature"] == "Leaf":
                leaf_value = tree_df.iloc[0]["Gain"]
                all_tree_formulas.append(And(o == leaf_value))
                continue
            path_formulas = []

            def get_conditions(node_id):
                conditions = []
                current_node = tree_df[tree_df["ID"] == node_id]
                if current_node.empty:
                    return conditions

                parent_node = tree_df[
                    (tree_df["Yes"] == node_id) | (tree_df["No"] == node_id)
                ]
                if not parent_node.empty:
                    parent_data = parent_node.iloc[0]
                    feature = parent_data["Feature"]
                    split_value = parent_data["Split"]
                    x = Real(feature)
                    if parent_data["Yes"] == node_id:
                        conditions.append(x < split_value)
                    else:
                        conditions.append(x >= split_value)
                    conditions = get_conditions(parent_data["ID"]) + conditions

                return conditions

            for _, node in tree_df[tree_df["Feature"] == "Leaf"].iterrows():
                leaf_value = node["Gain"]
                leaf_id = node["ID"]
                conditions = get_conditions(leaf_id)
                path_formula = And(*conditions)
                implication = Implies(path_formula, o == leaf_value)
                path_formulas.append(implication)

            all_tree_formulas.append(And(*path_formulas))
        return And(*all_tree_formulas)

    def get_init_value(self, model, x, estimator_variables):
        estimator_pred = Solver()
        estimator_pred.add(self.I)
        estimator_pred.add(self.T)
        if estimator_pred.check() == sat:
            solvermodel = estimator_pred.model()
            total_sum = sum(
                float(solvermodel.eval(var).as_fraction()) for var in estimator_variables
            )
        else:
            total_sum = 0
            print("estimator error")
        self.predicted_margin = model.predict(x, output_margin=True)[0]
        init_value = self.predicted_margin - total_sum
        self.init_value = init_value
        return init_value

    def decision_function_expression(self, model, x):
        n_classes = 1 if model.n_classes_ <= 2 else model.n_classes_
        predicted_class = model.predict(x)[0]
        self.predicted_class = predicted_class
        n_estimators = int(len(model.get_booster().get_dump()) / n_classes)
        estimator_variables = [Real(f"o_{i}_0") for i in range(n_estimators)] # _0 only for binary classification
        self.estimator_variables = estimator_variables
        init_value = self.get_init_value(model, x, estimator_variables)
        # print("init:", round(init_value, 2))

        equation_list = []

        estimator_sum = Real("estimator_sum")
        equation_o = estimator_sum == Sum(estimator_variables)
        equation_list.append(equation_o)

        decision = Real("decision")
        equation_list.append(decision == estimator_sum + init_value)

        if predicted_class == 0:
            final_equation = decision < 0
        else:
            final_equation = decision > 0

        return final_equation, And(equation_list)

    def instance_expression(self, instance):
        formula = [Real(self.columns[i]) == value for i, value in enumerate(instance)]
        return formula

    def explain_expression(self, I, T_s, D_s, model, reorder):
        i_expression = I.copy()

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]

        for feature in i_expression.copy():
            # print("\n---removed", feature)
            i_expression.remove(feature)

            # prove(Implies(And(And(i_expression), T), D))
            if self.is_proved(Implies(And(And(i_expression), T_s), D_s)):
                continue
                # print('proved')
            else:
                # print('not proved')
                i_expression.append(feature)
        # print(self.is_proved(Implies(And(And(i_expression), T_s), D_s)))
        return i_expression

    def explain_expression_prob(self, I, T_s, D_s, model, reorder, threshold_margin, target_threshold):
        i_expression = I.copy()

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]

        threshold = 0

        if target_threshold:
            threshold = target_threshold
        elif threshold_margin != 0:
            threshold = self.predicted_margin * threshold_margin/100
            # print("margin:", self.predicted_margin, "accepted margin:", threshold)
        self.xai_predicted_margin = self.predicted_margin

        for feature in i_expression.copy():
            # print("\n---removed", feature)
            i_expression.remove(feature)

            if self.is_proved_sat(And(And(i_expression), T_s), threshold):
                # print('proved')
                continue
            else:
                # print('not proved -- added back')
                i_expression.append(feature)
        return i_expression


    def is_proved(self, decision_exp):
        s = Solver()
        s.add(Not(decision_exp))
        if s.check() == unsat:
            return True
        else:
            # print(s.model())
            return False

    def is_proved_sat(self, decision_exp, threshold):
      decision = Real("decision")

      debug = Real("debug") == 0
      predicted_class = self.predicted_class

      if predicted_class == 0:
        estmax = Optimize()
        estmax.add(decision_exp)
        estmax.add(debug)
        maxvalue = estmax.maximize(decision)
        if estmax.check() == sat:
            # print("\nmax sat", maxvalue.value())
            try:
              if float(maxvalue.value().as_fraction()) > threshold:
                  return False # can change class
              else:
                  self.xai_predicted_margin = float(maxvalue.value().as_fraction())
            except:
              print("error max =", maxvalue.value())
              return False
        else:
            print("error")

      if predicted_class == 1:
        estmin = Optimize()
        estmin.add(decision_exp)
        estmin.add(debug)
        minvalue = estmin.minimize(decision)
        if estmin.check() == sat:
            # print("\nmin sat", minvalue.value())
            try:
              if float(minvalue.value().as_fraction()) < threshold:
                  return False # can change class
              else:
                  self.xai_predicted_margin = float(minvalue.value().as_fraction())
            except:
              print("error min =", minvalue.value())
              return False
        else:
            print("error")

      if predicted_class == 0:
        self.solvermodel = estmax.model()
      if predicted_class == 1:
        self.solvermodel = estmin.model()
      return True
  
def generate_results(explainer, X_test, y_pred, classification, path, reorder="asc"):
    results = []
    if classification == 0:
        increase_prob = -0.01
    else:
        increase_prob = 0.01
    for i in X_test[y_pred == classification].index:
        sample = X_test.loc[i].values
        xai = explainer.explain_prob(sample, reorder=reorder)
        xaiprob_initial = explainer.xai_predicted_margin
        len_xai_initial = len(xai)

        xai = explainer.explain_prob(sample, reorder=reorder, target_threshold=xaiprob_initial + increase_prob)
        xaiprob_final = explainer.xai_predicted_margin
        len_xai_final = len(xai)

        if round(xaiprob_initial, 2) == round(xaiprob_final, 2):
            len_xai_initial = len_xai_final
        results.append({
            "index": i,
            "class": classification,
            "xaiprob_initial": round(xaiprob_initial, 2),
            "len_xai_initial": len_xai_initial,
            "xaiprob_final": round(xaiprob_final, 2),
            "len_xai_final": len_xai_final
        })
    df_results = pd.DataFrame(results)
    return df_results
    # df_results.to_csv(f'{path}/results_{classification}_{reorder}.csv', index=False) 

# Prepare datasets and models

In [3]:
def prepare_dataset_model_explainer(dataset_name, dataset_params):
    # Fetch data
    dataset = fetch_data(dataset_name)
    X = dataset.drop('target', axis=1)
    y = dataset['target']
    
    if dataset_name == "adult" and len(X) > 5000:
        X, _, y, _ = train_test_split(
            X, y, train_size=5000, stratify=y, random_state=42
        )

    params = dataset_params[dataset_name]

    # Train model
    model = XGBClassifier(**params)
    model.fit(X, y)
    y_pred = model.predict(X)

    # Metrics
    print(f"== {dataset_name.upper()} Dataset ==")
    print("Dataset size:", len(X))
    print("Columns:", len(X.columns))
    # print("Accuracy:", accuracy_score(y, y_pred))
    # print("Precision:", precision_score(y, y_pred))
    # print("Recall:", recall_score(y, y_pred))
    print("F1-score:", f1_score(y, y_pred))

    # Explainer
    explainer = XGBoostExplainer(model, X)

    return model, X, y, explainer

def prepare_all_datasets(dataset_names, dataset_params):
    context = {}
    for name in dataset_names:
        print(f"\n--- Preparing {name} ---")
        model, X, y, explainer = prepare_dataset_model_explainer(name, dataset_params)
        context[name] = (model, X, y, explainer)
    return context

In [4]:
dataset_names = ["saheart", "adult", "mushroom", "sonar"]

dataset_params = {
    "adult": {"n_estimators": 30},
    "saheart": {"n_estimators": 30},
    "mushroom": {"n_estimators": 30},
    "sonar": {"n_estimators": 30},
}

dataset_context = prepare_all_datasets(dataset_names, dataset_params)


--- Preparing saheart ---
== SAHEART Dataset ==
Dataset size: 462
Columns: 9
F1-score: 0.9968652037617555

--- Preparing adult ---
== ADULT Dataset ==
Dataset size: 5000
Columns: 14
F1-score: 0.9409646302250804

--- Preparing mushroom ---
== MUSHROOM Dataset ==
Dataset size: 8124
Columns: 22
F1-score: 1.0

--- Preparing sonar ---
== SONAR Dataset ==
Dataset size: 208
Columns: 60
F1-score: 1.0


# Check explanations

In [5]:
def get_explain_results(dataset_name):
    model, X, y, explainer = dataset_context[dataset_name]
    y_pred = model.predict(X)

    # Function to explain a sample of a given class
    def explain_for_class(classification):
        # Get a sample of the specified class in predictions
        indices = X[y_pred == classification].index
        if len(indices) == 0:
            print(f"No samples predicted as class {classification}.")
            return
        i = indices[0]
        sample = X.loc[i].values

        # Initial explanation
        margin = model.predict([sample], output_margin=True)[0]
        print(f"\nClass {classification} - Margin: {margin}, Sample index: {i}")

        xai = explainer.explain_prob(sample)
        print("Initial abductive explanation:", xai)
        
        xaiprob_initial = explainer.xai_predicted_margin
        print("Initial predicted margin:", xaiprob_initial)

        # Adjust margin direction
        # increase_prob = -0.01 if classification == 0 else 0.01

        # Explanation with adjusted threshold
        xai_confidence = explainer.explain_prob(sample, target_threshold= margin / 2)
        xaiprob_final = explainer.xai_predicted_margin
        print("Confidence-aware abductive explanation:", xai_confidence)
        print("Final predicted margin:", xaiprob_final)

    # Explain for both classes
    explain_for_class(0)
    explain_for_class(1)


In [6]:
get_explain_results("mushroom")


Class 0 - Margin: -8.39293384552002, Sample index: 0
Initial abductive explanation: [gill-size == 0, spore-print-color == 1]
Initial predicted margin: -5.9726229332
Confidence-aware abductive explanation: [gill-size == 0, spore-print-color == 1]
Final predicted margin: -5.9726229332

Class 1 - Margin: 8.46451473236084, Sample index: 3
Initial abductive explanation: [stalk-surface-below-ring == 2, gill-spacing == 0, veil-color == 2, spore-print-color == 3]
Initial predicted margin: 2.37855409442
Confidence-aware abductive explanation: [stalk-surface-below-ring == 2, odor == 4, gill-spacing == 0, veil-color == 2, spore-print-color == 3]
Final predicted margin: 5.08424244958


In [7]:
get_explain_results("adult")


Class 0 - Margin: -5.76040506362915, Sample index: 33044
Initial abductive explanation: [age == 46, education-num == 13, capital-gain == 27828, relationship == 1]
Initial predicted margin: -1.5279432290741
Confidence-aware abductive explanation: [occupation == 4, age == 46, capital-loss == 0, education-num == 13, capital-gain == 27828, marital-status == 0, relationship == 1]
Final predicted margin: -3.02649918073

Class 1 - Margin: 1.490314245223999, Sample index: 47519
Initial abductive explanation: [fnlwgt == 174373, hours-per-week == 30, capital-loss == 0, education-num == 9, capital-gain == 0, relationship == 0]
Initial predicted margin: 0.009739591496
Confidence-aware abductive explanation: [fnlwgt == 174373, native-country == 39, hours-per-week == 30, capital-loss == 0, education-num == 9, capital-gain == 0, relationship == 0]
Final predicted margin: 1.060730009926


In [8]:
get_explain_results("sonar")


Class 0 - Margin: -5.715680122375488, Sample index: 0
Initial abductive explanation: [A17 == 0.287, A47 == 0.1688, A9 == 0.1865, A52 == 0.012, A48 == 0.1037, A51 == 0.013, A27 == 0.9301, A12 == 0.3553, A15 == 0.178, A4 == 0.057, A21 == 0.7969, A16 == 0.2794, A11 == 0.3188]
Initial predicted margin: -0.08802267005
Confidence-aware abductive explanation: [A18 == 0.3969, A8 == 0.1684, A37 == 0.0702, A55 == 0.0062, A17 == 0.287, A49 == 0.0501, A47 == 0.1688, A28 == 0.9955, A31 == 0.3934, A9 == 0.1865, A6 == 0.1091, A52 == 0.012, A48 == 0.1037, A51 == 0.013, A27 == 0.9301, A12 == 0.3553, A15 == 0.178, A4 == 0.057, A21 == 0.7969, A16 == 0.2794, A34 == 0.114, A11 == 0.3188]
Final predicted margin: -2.95120893735

Class 1 - Margin: 5.48876953125, Sample index: 12
Initial abductive explanation: [A47 == 0.0342, A28 == 0.6742, A54 == 0.0052, A9 == 0.1151, A48 == 0.0469, A45 == 0.084, A12 == 0.1102, A4 == 0.0139, A21 == 0.2005, A16 == 0.2138, A11 == 0.1203]
Initial predicted margin: 0.1394765883


In [9]:
get_explain_results("saheart")


Class 0 - Margin: -3.512935161590576, Sample index: 2
Initial abductive explanation: [Sbp == 118, Ldl == 3.48, Typea == 52, Tobacco == 0.08, Age == 46]
Initial predicted margin: -0.2604648052
Confidence-aware abductive explanation: [Obesity == 29.14, Ldl == 3.48, Adiposity == 32.28, Typea == 52, Tobacco == 0.08, Age == 46]
Final predicted margin: -2.44854886201

Class 1 - Margin: 2.5211021900177, Sample index: 0
Initial abductive explanation: [Sbp == 160, Ldl == 5.73, Adiposity == 23.11, Typea == 49, Tobacco == 12, Famhist == 1, Age == 52]
Initial predicted margin: 0.30301121976
Confidence-aware abductive explanation: [Obesity == 25.3, Sbp == 160, Ldl == 5.73, Typea == 49, Tobacco == 12, Famhist == 1, Age == 52]
Final predicted margin: 1.936349560668


# Check threshold datasets

## functions

In [10]:
def get_explain_results_n(dataset_name, n_samples):
    model, X, y, explainer = dataset_context[dataset_name]
    y_pred = model.predict(X)

    # Prepare result DataFrame
    results = []

    # Loop over both classes
    for classification in [0, 1]:
        # Get indices for predicted samples of this class
        indices = X[y_pred == classification].index[:n_samples]
        if len(indices) == 0:
            print(f"No samples predicted as class {classification}.")
            continue

        for idx in indices:
            sample = X.loc[idx].values

            # Initial explanation
            margin = model.predict([sample], output_margin=True)[0]
            xai_initial = explainer.explain_prob(sample)
            xaiprob_initial = explainer.xai_predicted_margin
            exp_len_initial = len(xai_initial)

            # Explanation with adjusted threshold (half the margin)
            xai_confidence_case1 = explainer.explain_prob(sample, target_threshold=margin / 2)
            xaiprob_case1 = explainer.xai_predicted_margin
            exp_len_case1 = len(xai_confidence_case1)
            
            increase_prob = -0.01 if margin < 0 else 0.01
            xai_confidence_case2 = explainer.explain_prob(sample, target_threshold= xaiprob_initial + increase_prob)
            xaiprob_case2 = explainer.xai_predicted_margin
            exp_len_case2 = len(xai_confidence_case2)

            # Append to results
            results.append({
                'sample_id': idx,
                'class': classification,
                'pred_margin': round(margin, 4),
                'exp_len_inicial': exp_len_initial,
                'xaiprob_inicial': round(xaiprob_initial, 4),
                'exp_len_case1': exp_len_case1,
                'xaiprob_case1': round(xaiprob_case1, 4),
                'exp_len_case2': exp_len_case2,
                'xaiprob_case2': round(xaiprob_case2, 4),
            })

    # Convert to DataFrame
    df_results = pd.DataFrame(results)
    
    return df_results

def summarize_explanations(df_results):
    # Agrupar por classe e calcular as métricas
    summary = df_results.groupby('class').agg(
        pred_margin_mean=('pred_margin', 'mean'),
        pred_margin_std=('pred_margin', 'std'),
        exp_len_inicial_mean=('exp_len_inicial', 'mean'),
        exp_len_inicial_std=('exp_len_inicial', 'std'),
        xaiprob_inicial_mean=('xaiprob_inicial', 'mean'),
        xaiprob_inicial_std=('xaiprob_inicial', 'std'),
        exp_len_case1_mean=('exp_len_case1', 'mean'),
        exp_len_case1_std=('exp_len_case1', 'std'),
        xaiprob_case1_mean=('xaiprob_case1', 'mean'),
        xaiprob_case1_std=('xaiprob_case1', 'std'),
        exp_len_case2_mean=('exp_len_case2', 'mean'),
        exp_len_case2_std=('exp_len_case2', 'std'),
        xaiprob_case2_mean=('xaiprob_case2', 'mean'),
        xaiprob_case2_std=('xaiprob_case2', 'std'),
    ).reset_index()

    # Função para combinar mean ± std com 2 casas decimais
    def format_mean_std(mean, std):
        return f"{mean:.2f} ± {std:.2f}"

    # Aplicar a formatação em cada par de colunas
    formatted = pd.DataFrame()
    formatted['class'] = summary['class']
    formatted['Pred Margin'] = summary.apply(lambda row: format_mean_std(row['pred_margin_mean'], row['pred_margin_std']), axis=1)
    formatted['Exp Len Inicial'] = summary.apply(lambda row: format_mean_std(row['exp_len_inicial_mean'], row['exp_len_inicial_std']), axis=1)
    formatted['ECM Inicial'] = summary.apply(lambda row: format_mean_std(row['xaiprob_inicial_mean'], row['xaiprob_inicial_std']), axis=1)
    formatted['Exp Len case1'] = summary.apply(lambda row: format_mean_std(row['exp_len_case1_mean'], row['exp_len_case1_std']), axis=1)
    formatted['ECM case1'] = summary.apply(lambda row: format_mean_std(row['xaiprob_case1_mean'], row['xaiprob_case1_std']), axis=1)
    formatted['Exp Len case2'] = summary.apply(lambda row: format_mean_std(row['exp_len_case2_mean'], row['exp_len_case2_std']), axis=1)
    formatted['ECM case2'] = summary.apply(lambda row: format_mean_std(row['xaiprob_case2_mean'], row['xaiprob_case2_std']), axis=1)

    return formatted


# datasets

In [11]:
df_saheart = get_explain_results_n("saheart", 100)
df_summary_saheart = summarize_explanations(df_saheart)
df_summary_saheart

,class,Pred Margin,Exp Len Inicial,ECM Inicial,Exp Len case1,ECM case1,Exp Len case2,ECM case2
0,0,-3.12 ± 1.17,5.71 ± 1.20,-0.36 ± 0.33,7.07 ± 0.79,-1.92 ± 0.66,6.24 ± 1.18,-0.79 ± 0.46
1,1,2.07 ± 0.87,6.46 ± 1.09,0.33 ± 0.27,7.26 ± 0.88,1.33 ± 0.51,6.94 ± 1.08,0.76 ± 0.35


In [12]:
df_adult = get_explain_results_n("adult", 100)
df_summary_adult = summarize_explanations(df_adult)
df_summary_adult

,class,Pred Margin,Exp Len Inicial,ECM Inicial,Exp Len case1,ECM case1,Exp Len case2,ECM case2
0,0,-1.99 ± 2.06,9.67 ± 3.28,-0.26 ± 0.31,10.76 ± 2.45,-1.16 ± 1.10,10.24 ± 3.16,-0.53 ± 0.48
1,1,3.22 ± 2.02,6.65 ± 1.39,0.20 ± 0.22,8.26 ± 1.11,1.80 ± 1.09,7.38 ± 1.83,0.56 ± 0.41


In [13]:
df_mushroom = get_explain_results_n("mushroom", 100)
df_summary_mushroom = summarize_explanations(df_mushroom)
df_summary_mushroom

,class,Pred Margin,Exp Len Inicial,ECM Inicial,Exp Len case1,ECM case1,Exp Len case2,ECM case2
0,0,-7.82 ± 0.95,2.11 ± 0.40,-5.29 ± 1.56,2.33 ± 0.82,-5.52 ± 1.01,3.11 ± 0.40,-6.19 ± 1.75
1,1,7.97 ± 0.80,4.51 ± 0.56,2.41 ± 0.63,5.44 ± 0.69,5.00 ± 0.69,5.51 ± 0.56,3.27 ± 0.48


In [14]:
df_sonar = get_explain_results_n("sonar", 100)
df_summary_sonar = summarize_explanations(df_sonar)
df_summary_sonar

,class,Pred Margin,Exp Len Inicial,ECM Inicial,Exp Len case1,ECM case1,Exp Len case2,ECM case2
0,0,-4.14 ± 0.90,14.62 ± 3.05,-0.05 ± 0.04,19.21 ± 2.83,-2.12 ± 0.45,14.83 ± 3.00,-0.13 ± 0.07
1,1,3.92 ± 0.95,13.36 ± 3.32,0.07 ± 0.07,17.55 ± 2.86,2.02 ± 0.50,13.76 ± 3.15,0.16 ± 0.12


# Test robustness

In [26]:
def get_robustness_results_n(dataset_name, n_samples, n_extra_samples, target_class=1,
                              mult_margin=0, noise_level=0.5):
    model, X, y, explainer = dataset_context[dataset_name]

    # Armazena os resultados em lista
    results = []

    count = 0
    for i in range(len(X)):
        if count >= n_samples:
            break

        sample = X.iloc[i]
        pred_class = model.predict([sample])[0]
        if pred_class != target_class:
            continue

        margin = model.predict([sample], output_margin=True)[0]
        explanation = explainer.explain_prob(sample, reorder="asc", target_threshold=margin * mult_margin)

        satisfying_df = generate_samples_for_conditions(X, explanation, n_samples=n_extra_samples, random_state=42)
        noisy_df = apply_noise_to_samples(X, satisfying_df, noise_level=noise_level)
        noisy_pred = model.predict(noisy_df)
        noisy_pred_margin = model.predict(noisy_df, output_margin=True)

        class_0 = np.sum(noisy_pred == 0)
        class_1 = np.sum(noisy_pred == 1)

        results.append({
            "amostra": i,
            "classe_prevista": int(pred_class),
            "classe_0_com_ruido": int(class_0),
            "classe_1_com_ruido": int(class_1),
            # "output_margin_medio": float(np.mean(noisy_pred_margin)),
            # "output_margin_max": float(np.max(noisy_pred_margin)),
            # "output_margin_min": float(np.min(noisy_pred_margin))
        })

        count += 1

    # Converte para DataFrame
    results_df = pd.DataFrame(results)

    # Cálculo de média ± desvio padrão para cada coluna
    mean_0 = results_df["classe_0_com_ruido"].mean()
    std_0 = results_df["classe_0_com_ruido"].std()

    mean_1 = results_df["classe_1_com_ruido"].mean()
    std_1 = results_df["classe_1_com_ruido"].std()

    mean_results = pd.DataFrame([{
        "explanation class": int(pred_class),
        "noise samples": n_extra_samples,
        "noise level": noise_level,
        "classified 0": f"{mean_0:.2f} ± {std_0:.2f}",
        "classified 1": f"{mean_1:.2f} ± {std_1:.2f}"
    }])

    return mean_results


def generate_samples_for_conditions(df, conditions, n_samples=10, random_state=None):
    """
    Gera amostras aleatórias que atendem a um conjunto de condições, variando apenas as outras features.
    """
    np.random.seed(random_state)
    fixed_values = {str(cond.arg(0)): float(cond.arg(1).as_fraction()) for cond in conditions}

    df_variation = df.drop(columns=fixed_values.keys())
    samples = {col: np.random.uniform(df[col].min(), df[col].max(), n_samples) for col in df_variation.columns}
    samples = {key: np.round(value, 2) for key, value in samples.items()}

    for feature, value in fixed_values.items():
        samples[feature] = [value] * n_samples

    generated_df = pd.DataFrame(samples)
    return generated_df[df.columns]


def apply_noise_to_samples(X, df, noise_level):
    df_noisy = df.copy()
    for col in df.columns:
        min_val = X[col].min()
        max_val = X[col].max()
        random_vals = np.random.uniform(min_val, max_val, size=len(df))
        # Interpola entre valor original e valor aleatório
        df_noisy[col] = (1 - noise_level) * df[col] + noise_level * random_vals
    return df_noisy

In [27]:
configs = [
    {"target_class": 0, "mult_margin": 0},
    {"target_class": 0, "mult_margin": 0.5},
    {"target_class": 1, "mult_margin": 0},
    {"target_class": 1, "mult_margin": 0.5},
]

In [32]:
all_results = []
for config in configs:
    df = get_robustness_results_n(
        "saheart", 
        n_samples=100, 
        n_extra_samples=100, 
        target_class=config["target_class"], 
        mult_margin=config["mult_margin"], 
        noise_level=0.1
    )

    df["exp. threshold"] = "0%" if config["mult_margin"] == 0 else "50%"
    all_results.append(df)

results_df = pd.concat(all_results, ignore_index=True)
results_df

,explanation class,noise samples,noise level,classified 0,classified 1,exp. threshold
0,0,100,0.1,86.76 ± 13.82,13.24 ± 13.82,0%
1,0,100,0.1,88.69 ± 15.60,11.31 ± 15.60,50%
2,1,100,0.1,57.27 ± 30.01,42.73 ± 30.01,0%
3,1,100,0.1,53.67 ± 32.62,46.33 ± 32.62,50%


In [ ]:
all_results = []
for config in configs:
    df = get_robustness_results_n(
        "adult", 
        n_samples=100,
        n_extra_samples=100, 
        target_class=config["target_class"], 
        mult_margin=config["mult_margin"], 
        noise_level=0.1
    )

    df["target_class"] = config["target_class"]
    df["exp. threshold"] = "0%" if config["mult_margin"] == 0 else "50%"
    all_results.append(df)

results_df = pd.concat(all_results, ignore_index=True)
results_df

In [ ]:
all_results = []
for config in configs:
    df = get_robustness_results_n(
        "mushroom", 
        n_samples=100,
        n_extra_samples=100, 
        target_class=config["target_class"], 
        mult_margin=config["mult_margin"], 
        noise_level=0.1
    )

    df["target_class"] = config["target_class"]
    df["exp. threshold"] = "0%" if config["mult_margin"] == 0 else "50%"
    all_results.append(df)

results_df = pd.concat(all_results, ignore_index=True)
results_df

In [ ]:
all_results = []
for config in configs:
    df = get_robustness_results_n(
        "sonar", 
        n_samples=100,
        n_extra_samples=100, 
        target_class=config["target_class"], 
        mult_margin=config["mult_margin"], 
        noise_level=0.1
    )

    df["target_class"] = config["target_class"]
    df["exp. threshold"] = "0%" if config["mult_margin"] == 0 else "50%"
    all_results.append(df)

results_df = pd.concat(all_results, ignore_index=True)
results_df